Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [2]:
from random import choice, randint, random
from dataclasses import dataclass
from copy import copy
import sys
from tqdm import tqdm

import lab9_lib

## Problem Initialization

In [11]:
POPULATION_SIZE = 30
OFFSPRING_SIZE = 60
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .2
LOCI = 1000
PROBLEM = 1
GENERATIONS = 10000
MIGRANTS = 10
ISLANDS = 8
MIGRATE = 100

fitness = lab9_lib.make_problem(PROBLEM)

In [4]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, LOCI-1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, LOCI-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == LOCI
    return offspring

def uniform_crossover(ind1: Individual, ind2: Individual) -> Individual:
    offspring = Individual(fitness=None,
                           genotype=[choice([ind1.genotype[i], ind2.genotype[i]])
                                     for i in range(LOCI)])
    assert len(offspring.genotype) == LOCI
    return offspring

In [5]:
class Island:
    population: list[Individual]
    id: int

    def __init__(self, id):
        self.population = [
            Individual(
                genotype=[choice((True, False)) for _ in range(LOCI)],
                fitness=None,
            )
            for _ in range(POPULATION_SIZE)
        ]
        self.id = id

        for i in self.population:
            i.fitness = fitness(i.genotype)

    def step(self, best_fitness) -> float:
        for _ in range(MIGRATE):
            offspring = list()
            for _ in range(OFFSPRING_SIZE):
                if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
                    # mutation  # add more clever mutations
                    p = select_parent(self.population)
                    o = mutate(p)
                else:
                    # xover # add more xovers
                    p1 = select_parent(self.population)
                    p2 = select_parent(self.population)
                    # o = one_cut_xover(p1, p2)
                    o = uniform_crossover(p1, p2)
                offspring.append(o)

            for i in offspring:
                i.fitness = fitness(i.genotype)
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:POPULATION_SIZE]
    
            if self.population[0].fitness > best_fitness:
                best_fitness = self.population[0].fitness
                print("Island ", self.id, ": ", best_fitness)
        return best_fitness

    def inject(self, migrants: list[Individual]):
        self.population.extend(migrants)

    def migrate(self) -> list[Individual]:
        migrants = sorted(self.population, key=lambda x: x.fitness, reverse=True)[:MIGRANTS]
        return migrants

In [63]:
# fitness = lab9_lib.make_problem(PROBLEM)
'''for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

population = [
    Individual(
        genotype=[choice((True, False)) for _ in range(LOCI)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)'''

'for n in range(10):\n    ind = choices([0, 1], k=50)\n    print(f"{\'\'.join(str(g) for g in ind)}: {fitness(ind):.2%}")\n\nprint(fitness.calls)\n\npopulation = [\n    Individual(\n        genotype=[choice((True, False)) for _ in range(LOCI)],\n        fitness=None,\n    )\n    for _ in range(POPULATION_SIZE)\n]\n\nfor i in population:\n    i.fitness = fitness(i.genotype)'

## GA execution

In [1]:
'''best_fitness = 0
for generation in tqdm(range(GENERATIONS), file=sys.stdout):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    
    if population[0].fitness > best_fitness:
        best_fitness = population[0].fitness
        print(" ", best_fitness)'''
    

'best_fitness = 0\nfor generation in tqdm(range(GENERATIONS), file=sys.stdout):\n    offspring = list()\n    for counter in range(OFFSPRING_SIZE):\n        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability\n            # mutation  # add more clever mutations\n            p = select_parent(population)\n            o = mutate(p)\n        else:\n            # xover # add more xovers\n            p1 = select_parent(population)\n            p2 = select_parent(population)\n            o = one_cut_xover(p1, p2)\n        offspring.append(o)\n\n    for i in offspring:\n        i.fitness = fitness(i.genotype)\n    population.extend(offspring)\n    population.sort(key=lambda i: i.fitness, reverse=True)\n    population = population[:POPULATION_SIZE]\n    \n    if population[0].fitness > best_fitness:\n        best_fitness = population[0].fitness\n        print(" ", best_fitness)'

## The Island method

In [12]:
islands = []
for id in range(ISLANDS):
    i = Island(id)
    islands.append(i)

best_fitness = 0
g = 0
while g < GENERATIONS // MIGRATE:
    print("STEP: ", g)
    for i in range(ISLANDS):
        print("ISLAND: ", i)
        best_fitness = islands[i].step(best_fitness)
        if best_fitness == 1.0:
            exit
    migrants = []
    for i in range(ISLANDS):
        migrants.append(islands[i].migrate())
    for j in range(ISLANDS):
        for k in range(ISLANDS):
            if j == k: continue
            islands[j].inject(migrants[k])
    g+=1


STEP:  0
ISLAND:  0
Island  0 :  0.545
Island  0 :  0.546
Island  0 :  0.57
Island  0 :  0.573
Island  0 :  0.582
Island  0 :  0.586
Island  0 :  0.599
Island  0 :  0.612
Island  0 :  0.623
Island  0 :  0.635
Island  0 :  0.641
Island  0 :  0.642
Island  0 :  0.65
Island  0 :  0.654
Island  0 :  0.664
Island  0 :  0.671
Island  0 :  0.674
Island  0 :  0.683
Island  0 :  0.688
Island  0 :  0.691
Island  0 :  0.694
Island  0 :  0.699
Island  0 :  0.701
Island  0 :  0.706
Island  0 :  0.709
Island  0 :  0.714
Island  0 :  0.716
Island  0 :  0.718
Island  0 :  0.722
Island  0 :  0.727
Island  0 :  0.729
Island  0 :  0.731
Island  0 :  0.734
Island  0 :  0.736
Island  0 :  0.738
Island  0 :  0.74
Island  0 :  0.742
Island  0 :  0.745
Island  0 :  0.746
Island  0 :  0.749
Island  0 :  0.751
Island  0 :  0.753
Island  0 :  0.754
Island  0 :  0.755
Island  0 :  0.756
Island  0 :  0.757
Island  0 :  0.758
Island  0 :  0.759
Island  0 :  0.76
Island  0 :  0.761
Island  0 :  0.762
Island  0 :  0.

KeyboardInterrupt: 